In [ ]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

# Load data

In [ ]:
vacancies_df = pd.read_csv("vacancy_5.csv", sep="|")

<ipython-input-53-6910981cc5f6>:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  vacancies_df = pd.read_csv("vacancy_5.csv", sep="|")


In [ ]:
cv_df = pd.read_csv("dst-3.0_16_1_hh_database.csv", sep=";")

# Prepare vacancies df

In [ ]:
vacancies_df.columns

Index(['vacancy_name', 'busy_type', 'salary', 'education',
       'required_experience', 'languageKnowledge', 'hardSkills', 'softSkills'],
      dtype='object')

In [ ]:
vacancies_df = vacancies_df[['vacancy_name', 'busy_type', 'salary', 'education', 'required_experience', 'languageKnowledge', 'hardSkills', 'softSkills']]

In [ ]:
vacancies_df['hardSkills'] = vacancies_df['hardSkills'].apply(lambda x: ', '.join([y['hard_skill_name'] for y in json.loads(x)]))

In [ ]:
vacancies_df['softSkills'] = vacancies_df['softSkills'].apply(lambda x: ', '.join([y['soft_skill_name'] for y in json.loads(x)]))

In [ ]:
vacancies_df['languageKnowledge'] = vacancies_df['languageKnowledge'].apply(lambda x: ', '.join([y['code_language']+": "+y.get('level','') for y in json.loads(x)]))

In [ ]:
vacancies_df.head()

,vacancy_name,busy_type,salary,education,required_experience,languageKnowledge,hardSkills,softSkills
0,Машинист экскаватора,Полная занятость,от 23000,Не указано,0.0,,,
1,наладчик пакетоделательных машин,Полная занятость,от 70000,Не указано,0.0,,,
2,Сварщик газовой сварки,Полная занятость,от 60000,Среднее профессиональное,0.0,,,
3,Электрогазосварщик,Полная занятость,от 16242,Среднее профессиональное,0.0,,,
4,Водитель погрузчика,Полная занятость,от 53700,Не указано,1.0,,,


In [ ]:
vacancies_df['data'] = vacancies_df.apply(lambda row: ', '.join(['[' + c + '] ' + str(row[c]) for c in row.index if c not in ['vacancy_name']]), axis=1)

In [ ]:
vacancies_df = vacancies_df[['vacancy_name', 'data']]

In [ ]:
vacancies_df

,vacancy_name,data
29148,Инженер-проектировщик,"[busy_type] Полная занятость, [salary] от 3500..."
34786,ведущий методист,"[busy_type] Полная занятость, [salary] от 3000..."
67595,Специалист по работе с физическими лицами / Аб...,"[busy_type] Полная занятость, [salary] от 5700..."
39060,Ведущий специалист,"[busy_type] Полная занятость, [salary] от 2770..."
34165,Водитель автомобиля (пожарный автомобиль кат. С),"[busy_type] Полная занятость, [salary] от 0, [..."
...,...,...
53679,Электрогазосварщик,"[busy_type] Полная занятость, [salary] от 4000..."
65375,Следователь,"[busy_type] Полная занятость, [salary] от 5000..."
45960,Инженер,"[busy_type] Временная, [salary] от 55000, [edu..."
47808,Преподаватель,"[busy_type] Полная занятость, [salary] от 2761..."


# Prepare cv df

In [ ]:
cv_df.columns

Index(['Пол, возраст', 'ЗП', 'Ищет работу на должность:',
       'Город, переезд, командировки', 'Занятость', 'График', 'Опыт работы',
       'Последнее/нынешнее место работы', 'Последняя/нынешняя должность',
       'Образование и ВУЗ', 'Обновление резюме', 'Авто'],
      dtype='object')

In [ ]:
cv_df.columns = cv_df.columns.str.replace('ЗП', 'salary')
cv_df.columns = cv_df.columns.str.replace('ЗП', 'salary')
cv_df.columns = cv_df.columns.str.replace('Опыт работы', 'experience')
cv_df.columns = cv_df.columns.str.replace('Образование и ВУЗ', 'education')

In [ ]:
cv_df.head()

,"Пол, возраст",salary,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,experience,Последнее/нынешнее место работы,Последняя/нынешняя должность,education,Обновление резюме,Авто
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано
4,"Женщина , 26 лет , родилась 3 марта 1993",140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,22.04.2019 10:32,Не указано


In [ ]:
vacancies_df.to_csv('vac.csv', index=False)
cv_df.to_csv('cvs.csv', index=False)

# Load data with embeddings

In [ ]:
vac_df = pd.read_csv("vac.csv")

In [ ]:
vac_df.head()

,Unnamed: 0,vacancy_name,combined,embeddings
0,0,Инженер-проектировщик,"[busy_type] Полная занятость, [salary] от 3500...",[ 8.73220786e-02 -5.04226983e-02 -2.52846777e-...
1,1,ведущий методист,"[busy_type] Полная занятость, [salary] от 3000...",[ 8.36978108e-02 -5.42348698e-02 -2.78635472e-...
2,2,Специалист по работе с физическими лицами / Аб...,"[busy_type] Полная занятость, [salary] от 5700...",[ 6.63708448e-02 -5.69521412e-02 -2.18575988e-...
3,3,Ведущий специалист,"[busy_type] Полная занятость, [salary] от 2770...",[ 9.36675966e-02 -5.23465239e-02 -2.00755266e-...
4,4,Водитель автомобиля (пожарный автомобиль кат. С),"[busy_type] Полная занятость, [salary] от 0, [...",[ 9.09329951e-02 -3.99896614e-02 5.89775853e-...


In [ ]:
cv_df = pd.read_csv("cvs.csv")

In [ ]:
cv_df.head()

,Unnamed: 0,"Пол, возраст",salary,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,experience,Последнее/нынешнее место работы,Последняя/нынешняя должность,education,Обновление резюме,Авто,combined,embeddings
0,0,"Мужчина , 30 лет , родился 16 ноября 1988",60000 руб.,Сервисный инженер,"Улан-Удэ , не готов к переезду , не готов к ко...",полная занятость,полный день,Опыт работы 5 лет 1 месяц Май 2014 — по насто...,ау рб ркгвв,Инженер по медицинскому оборудованию,Высшее образование 2010 Восточно-Сибирский го...,06.04.2019 04:17,Имеется собственный автомобиль,"Мужчина , 30 лет , родился 16 ноября 1988\n60...","[0.0624009967, 0.0149446484, 0.0306787081, -0...."
1,1,"Мужчина , 31 год , родился 7 мая 1987",35000 руб.,Специалист по IT,"Чита , готов к переезду (Москва, Санкт-Петербу...","частичная занятость, полная занятость","гибкий график, полный день, сменный график, ва...",Опыт работы 7 лет 4 месяца Специалист по IT 3...,AO ZABTEK,Специалист по IT,Неоконченное высшее образование 2005 Санкт-Пе...,25.04.2019 09:15,Не указано,"Мужчина , 31 год , родился 7 мая 1987\n35000 ...","[0.104451291, -0.0410094634, 0.0523592345, -0...."
2,2,"Женщина , 36 лет , родилась 17 сентября 1982",35555 руб.,Менеджер по управлению проектами,"Тверь , готова к переезду , готова к командиро...",полная занятость,полный день,Опыт работы 7 лет 2 месяца Менеджер по управл...,"Актион-Медиа, ЗАО",Специалист по продажам,Высшее образование 2015 Учебный Центр КАМИН Се...,26.04.2019 17:44,Не указано,"Женщина , 36 лет , родилась 17 сентября 1982\...","[0.109999247, -0.0114596635, 0.0195585787, -0...."
3,3,"Мужчина , 36 лет , родился 31 августа 1982",120000 руб.,Программист,"Москва , готов к переезду (Россия) , готов к р...",полная занятость,удаленная работа,Опыт работы 17 лет 7 месяцев Программист 120 ...,"Роза Хутор, ООО",Главный специалист отдела информационных техно...,Высшее образование 2007 Тверская государственн...,22.11.2018 11:35,Не указано,"Мужчина , 36 лет , родился 31 августа 1982\n1...","[0.04130933, -0.0471919, 0.01153475, -0.020690..."
4,4,"Мужчина , 30 лет , родился 16 мая 1988",40000 руб.,Технический специалист,"Екатеринбург , не готов к переезду , не готов ...",полная занятость,полный день,Опыт работы 5 лет Ноябрь 2016 — по настоящее ...,Почта России,Ведущий специалист Helpdesk,Неоконченное высшее образование 2008 Филиал М...,11.04.2019 20:49,Не указано,"Мужчина , 30 лет , родился 16 мая 1988\n40000...","[0.0570270978, 0.0167246982, -0.00310655613, -..."


# Preprocess embeddings

In [ ]:
vac_df['embeddings'] = vac_df['embeddings'].apply(lambda x: np.array([float(val) for val in re.split('\s+', x.replace('[', '').replace(']', '').replace('\n', '')) if val]))
cv_df['embeddings'] = cv_df['embeddings'].apply(lambda x: np.array([float(val) for val in re.split('\s+', x.replace('[', '').replace(']', '').replace('\n', '')) if val]))

# Calculate matched vacancies

In [ ]:
vac_embeddings = np.array(vac_df['embeddings'].tolist())
cv_embeddings = np.array(cv_df['embeddings'].tolist())

similarity_matrix = cosine_similarity(cv_embeddings, vac_embeddings)

top_matches_indices = np.argsort(similarity_matrix, axis=1)[:, ::-1][:, :5]

result_df = pd.concat([
    pd.DataFrame({
        'cv_name': [cv_df.at[i, 'embeddings']] * 5,
        'matched_vacancies': vac_df.loc[row, 'embeddings'].tolist(),
        'similarity': (similarity_matrix[i, row]).tolist()
    })
    for i, row in enumerate(top_matches_indices)
], ignore_index=True)

In [ ]:
result_df

,cv_name,matched_vacancies,similarity
0,"[0.0624009967, 0.0149446484, 0.0306787081, -0....","[0.04075013, 0.00121289, -0.00365269, -0.06629...",0.673945
1,"[0.0624009967, 0.0149446484, 0.0306787081, -0....","[0.0923710316, 0.00726321852, -0.00585568091, ...",0.649069
2,"[0.0624009967, 0.0149446484, 0.0306787081, -0....","[0.109533951, 0.00295943138, -0.00501219742, -...",0.648216
3,"[0.0624009967, 0.0149446484, 0.0306787081, -0....","[0.075875245, -0.00378074055, 0.00611559162, -...",0.643945
4,"[0.0624009967, 0.0149446484, 0.0306787081, -0....","[0.0812830403, -0.0255423635, -0.00249464368, ...",0.643678
...,...,...,...
4995,"[0.056670703, 0.0475651771, 0.00702568842, -0....","[0.0923710316, 0.00726321852, -0.00585568091, ...",0.734127
4996,"[0.056670703, 0.0475651771, 0.00702568842, -0....","[0.0610071272, -0.0361962765, -0.0342499986, -...",0.717594
4997,"[0.056670703, 0.0475651771, 0.00702568842, -0....","[0.04075013, 0.00121289, -0.00365269, -0.06629...",0.701562
4998,"[0.056670703, 0.0475651771, 0.00702568842, -0....","[0.0812830403, -0.0255423635, -0.00249464368, ...",0.695021


In [ ]:
result_df.to_csv('result.csv', index=False)